In [1]:
from api_helper import ShoonyaApiPy
import logging
import time
import pandas as pd
import json
import numpy as np
import cred
from datetime import datetime
import time
from notify_run import Notify
notify = Notify()
import pyotp
token = cred.token
totp = pyotp.TOTP(token).now()
#enable dbug to see request and responses
logging.basicConfig(level=logging.DEBUG)
#start of our program
api = ShoonyaApiPy()
#credentials
user = cred.user
pwd = cred.pwd
factor2 = totp
vc = cred.vc
app_key = cred.apikey
imei = cred.imei
logins = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, 
    api_secret=app_key,imei = imei)
print(logins)

In [ ]:
n = pd.DataFrame(columns = ['p','v'])
bn = pd.DataFrame(columns = ['p','v'])
ni = bni = 0
buy_n = pd.DataFrame(columns = ['buy'])
buy_bn = pd.DataFrame(columns = ['buy'])
nb = bnb = 0
ret_n = ret_bn = 0
vol_n = vol_bn =0
ch_vn = ch_vbn = 0
sell_n = pd.DataFrame(columns = ['sell'])
sell_bn = pd.DataFrame(columns = ['sell'])
ns= bns = 0
in_traden = in_tradebn = False
signal = 0
feed_opened = False
sp_n = sp_bn = 0

In [ ]:

def event_handler_feed_update(m):
    global n,bn,ni,bni,buy_n,buy_bn,nb,bnb,ret_n,ret_bn,vol_n,vol_bn,ch_vn,ch_vbn,sell_n,sell_bn,ns,bns,in_traden,in_tradebn,signal,feed_opened, sp_n,sp_bn
    
    if(m['tk']==nifty):
        if('pc' in m):
            vol_n = float(m['pc'])
        if('lp' in m):
            sp_n = float(m['lp'])
            n.loc[ni] = [sp_n,vol_n]
            ni = ni+1
        if(ni>30):
            ret_n = (n['p'][ni-1] - n['p'][ni-21])
            ch_vn = (n['v'][ni-1] - n['v'][ni-21])/n['v'][ni-21]
            
        if(signal==1):
            buy_n.loc[nb] = sp_n
            nb = nb+1
            in_traden = True
            signal = 0
            #notify.send('buy nifty @' + str(sp_n))
        if(in_traden==True):
            if(sp_n>=buy_n['buy'][nb-1]+10):
                sell_n.loc[ns] = sp_n
                in_traden = False
                ns  = ns+1
               # notify.send('nifty target')
            elif(sp_n<=buy_n['buy'][nb-1]-5):
                sell_n.loc[ns] = sp_n
                in_traden = False
                ns = ns+1
              #  notify.send('nifty sl') 
        
        
    if(m['tk'] == banknifty):
        if('pc' in m):
            vol_bn = float(m['pc'])
        if('lp' in m ):
            sp_bn = float(m['lp'])
            bn.loc[bni] = [sp_bn, vol_bn]
            bni = bni+1
            
        if(bni > 30):
            ret_bn = (bn['p'][bni-1] - bn['p'][bni-21])
            ch_vbn = (bn['p'][bni-1] - bn['p'][bni-21])/bn['p'][bni-21]
            
        if(signal==2):
            buy_bn.loc[bnb] = sp_bn
            bnb = bnb +1
            in_tradebn = True
            signal = 0
            #notify.send('buy banknifty @' + str(sp_bn))
        if(in_tradebn == True):
            if(sp_bn>=buy_bn['buy'][bnb-1]+20):
                sell_bn.loc[bns] = sp_bn
                in_tradebn = False
                bns = bns+1
               # notify.send('banknifty target')
            elif(sp_bn<=buy_bn['buy'][bnb-1]-10):
                sell_bn.loc[bns] = sp_bn
                in_tradebn = False
                bns = bns+1
               # notify.send('banknifty sl')
            
            
                
                
                
    if((in_traden==False) and (in_tradebn==False)):
        if(ret_bn>0 and ret_n<0 ):
            signal = 1
        elif(ret_bn<0 and ret_n>0 ):
            signal = 2
        
    
        
def event_handler_order_update(inmessage):
    print(inmessage)                                      #     if(('norennordno' in inmessage)&('status'in inmessage)):  #         orderJson[inmessage['norenordno']] = {'status': inmessage['status']}
        
def open_callback():
    
    global feed_opened
    feed_opened = True



In [ ]:
api.start_websocket( order_update_callback=event_handler_order_update,
 subscribe_callback=event_handler_feed_update, 
socket_open_callback=open_callback)
while(feed_opened==False):
    pass


In [ ]:
# b = api.searchscrip('NSE','NIFTY INDEX')
# nifty = b['values'][0]['token']
# b = api.searchscrip('NSE','NIFTY BANK')
# banknifty = b['values'][0]['token']

In [ ]:
nifty = '26000'
banknifty = '26009'

In [ ]:
api.subscribe(['NSE|26000', 'NSE|26009'])

In [ ]:
buy_bn

In [ ]:
sell_bn

In [ ]:
buy_n

In [ ]:
sell_n

In [ ]:
df_n = pd.concat([buy_n,sell_n],axis = 1)
df_n.dropna(inplace =True)
df_n['net'] = df_n['sell']- df_n['buy']
nifty_pnl = df_n['net'].sum()*5000*0.05-100*df_n['net'].count()
nifty_pnl

In [ ]:
df_bn = pd.concat([buy_bn,sell_bn],axis = 1)
df_bn.dropna(inplace =True)
df_bn['net'] = df_bn['sell']- df_bn['buy']
banknifty_pnl = df_bn['net'].sum()*3500*0.05-100*df_bn['net'].count()
banknifty_pnl

In [ ]:
pnl=nifty_pnl+banknifty_pnl